In [2]:
import pandas as pd
import numpy as np
#import mysql.connector
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
#python -m nltk.downloader
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [3]:
# sql = """
# SELECT *  FROM `mimetic-coral-355913.dwh.comment`
# """
# df = client.query(sql).to_dataframe()
# df.head()

In [4]:
df = pd.read_csv("gs://the-clean-project/train.csv")
df.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [5]:
df['y'] = (df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0 ).astype(int)
df = df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})
df.sample(5)
#juste ça à sauvergarder et mettre dans une table

,text,y
38415,", GAY! Ha \n\nI thought the same thing, He mig...",1
69637,Support If only the intro of the article/list ...,0
43080,You don't want the truth?? She IS and WAS a tr...,1
85870,Looking for Alibrandi \n\nDid you edit Looking...,0
46010,"Deletion!?\nThis is pretty ridiculous. Dude, y...",0


In [6]:
df['y'].value_counts(normalize=True)
min_len = (df['y'] == 1).sum()
df_y0_undersample = df[df['y'] == 0].sample(n=min_len, random_state=201)
df = pd.concat([df[df['y'] == 1], df_y0_undersample])
df['y'].value_counts()

from nltk.stem import SnowballStemmer, WordNetLemmatizer
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
all_stopwords = stopwords.words('english')

def clean(comment):
    comment = re.sub('[^a-zA-Z]', ' ', comment)
    comment = comment.lower()
    comment = comment.split()
    comment = [stemmer.stem(word) for word in comment if not word in set(all_stopwords)]
    comment = [lemmatizer.lemmatize(word) for word in comment]
    comment = ' '.join(comment)
    return comment

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
df['text'].iloc[0]

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [8]:
clean(df['text'].iloc[0])

'cocksuck piss around work'

In [9]:
df['text'] = df['text'].apply(clean)

## TF-IDF

In [10]:
vec = TfidfVectorizer()

In [11]:
X = vec.fit_transform(df['text'])
X

<32450x45777 sparse matrix of type '<class 'numpy.float64'>'
	with 728904 stored elements in Compressed Sparse Row format>

## Fit Naive Bayes

In [12]:
model = MultinomialNB()
model.fit(X, df['y'])

MultinomialNB()

## Validate

In [13]:
# Le fichier validation est chargé dans le bucket dans le même dossier que train.csv !   ;-)
df_val = pd.read_csv("gs://the-clean-project/validation_data.csv")

X_less_toxic = vec.transform(df_val['less_toxic'].apply(clean))
X_more_toxic = vec.transform(df_val['more_toxic'].apply(clean))

print(X_less_toxic)
p1 = model.predict_proba(X_less_toxic)
p2 = model.predict_proba(X_more_toxic)

# Validation Accuracy
(p1[:, 1] < p2[:, 1]).mean()

  (0, 44677)	0.47325959475777485
  (0, 44630)	0.8492847949187935
  (0, 38435)	0.19945406729290457
  (0, 2353)	0.12230604289963462
  (1, 45217)	0.1732972786668851
  (1, 42408)	0.2038543852331415
  (1, 40970)	0.2603617671633731
  (1, 39991)	0.138357848726357
  (1, 37107)	0.14945796764236977
  (1, 34913)	0.12631417440535433
  (1, 33656)	0.2112521379260806
  (1, 32775)	0.2412313176583086
  (1, 29839)	0.12049540889814096
  (1, 28474)	0.11188204851365635
  (1, 28084)	0.1973077015249387
  (1, 24330)	0.5055414713954454
  (1, 19843)	0.3376846138406831
  (1, 17515)	0.21393466612428982
  (1, 14821)	0.3318844135768454
  (1, 14240)	0.14515846998219514
  (1, 13791)	0.227342985125694
  (1, 10639)	0.19006858443272837
  (2, 45490)	0.4427090058750191
  (2, 44724)	0.24080110831752685
  (2, 43825)	0.35152511917431084
  :	:
  (30105, 358)	0.3466708069922759
  (30106, 44770)	0.39287644977561187
  (30106, 42573)	0.26380622068501286
  (30106, 39170)	0.1962642106396804
  (30106, 36363)	0.32924660003261474
  (3

0.6675634382888269

## Application on our comments

### Importer les colonnes qui nous intéressent, à savoir ici : id_comment, content

In [14]:
sql_jig = """
SELECT id_comment, content  FROM `mimetic-coral-355913.dwh.comment`
"""
df_jig = client.query(sql_jig).to_dataframe()
df_jig.head()

NameError: name 'client' is not defined

### Faut sûrement nettoyer les contents avant d'appliquer le modèle ? (fonction clean ci-dessus)

In [ ]:
X_test = vec.transform(df_jig['content'])

### Appliquer le modèle sur nos données

In [ ]:
p3 = model.predict_proba(X_test)

In [ ]:
df_jig['score_jigsaw'] = p3[:,1]
del df_jig['content']
df_jig
# len(df_jig)

In [ ]:
sql_j = """
SELECT id_comment, type_content  FROM `mimetic-coral-355913.dwh.comment`
"""
df_j = client.query(sql_j).to_dataframe()
df_j.head()

In [ ]:
len(df_j)

In [ ]:
df_jig=df_jig.merge(df_j, left_on='id_comment', right_on='id_comment')
df_jig


## Tâche finale : enregistrer les scores ainsi calculés dans le DWH

In [ ]:
# dataset_id = 'dwh'
# # For this sample, the table must already exist and have a defined schema
# table_id ="score_jig"
# table_ref = client.dataset(dataset_id).table(table_id)
# table = client.get_table(table_ref)
# client.insert_rows(table, df_jig.values.tolist())


In [ ]:
#from google.cloud import bigquery

#client = bigquery.Client()

#dml_statement = ("UPDATE `dwh.test` SET colonne2 = 'A' WHERE colonne1 = 2;")
#print(dml_statement)
#query_job = client.query(dml_statement)  # API request
#query_job.result()  # Waits for statement to finish

In [ ]:
#pip install pandas-gbq

In [ ]:
import pandas_gbq

In [ ]:
pandas_gbq.to_gbq(df_jig, "dwh.score_jig", project_id="mimetic-coral-355913", if_exists="replace")

In [ ]:
# def bq_load(key, value):
  
#   project_name = 'YOUR PROJECT NAME'
#   dataset_name = 'YOUR DATASET NAME'
#   table_name = key
  
#   value.to_gbq(destination_table='{}.{}'.format(dataset_name, table_name), project_id=project_name, if_exists='replace')
# bq_load("mimetic-coral-355913.dwh.score_jig",df_jig)

In [ ]:
# from google.cloud import bigqueryclient = bigquery.Client()

# assert table.encryption_configuration.kms_key_name == original_kms_key_name

# # Set a new encryption key to use for the destination.
# # TODO: Replace this key with a key you have created in KMS.
# updated_kms_key_name = (
#     ""
# )
# table.encryption_configuration = bigquery.EncryptionConfiguration(
#     kms_key_name=updated_kms_key_name
# )

# table = client.update_table(table, ["encryption_configuration"])  # API request

# assert table.encryption_configuration.kms_key_name == updated_kms_key_name
# assert original_kms_key_name != updated_kms_key_name

In [ ]:
# sql = """
# SELECT *  FROM `mimetic-coral-355913.dwh.comment`
# """
# df15 = client.query(sql).to_dataframe()
# df15.head()